# miniGPT-v0.2

## Using context (Bag Of words)

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
from datetime import datetime

B,T,C = 4,8,2


In [4]:
x = torch.tensor(np.random.randn(B,T,C))
x.shape

torch.Size([4, 8, 2])

In [7]:
xbow = torch.zeros(B,T,C) 
for b in range(B):
    for t in range(T):
        prev_x = x[b, :t+1]
        xbow[b,t] = prev_x.mean(dim=0)
x[0]
xbow[0]

tensor([[-2.1126, -1.2052],
        [ 0.1902,  1.1745],
        [ 2.5900,  1.8854],
        [-0.3170,  0.8403],
        [-1.3186,  0.1055],
        [ 0.6145, -0.7667],
        [-0.8824, -0.1394],
        [ 1.8297, -0.2798]], dtype=torch.float64)

tensor([[-2.1126, -1.2052],
        [-0.9612, -0.0153],
        [ 0.2226,  0.6182],
        [ 0.0877,  0.6737],
        [-0.1936,  0.5601],
        [-0.0589,  0.3390],
        [-0.1765,  0.2706],
        [ 0.0742,  0.2018]])

In [10]:
multiplier = torch.zeros(T,T,dtype=torch.float64)

for t in range(T):
    for t2 in range(t+1):
        multiplier[t,t2] = 1 
multiplier 

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]], dtype=torch.float64)

In [18]:
mul = multiplier @ x[0] 
mul = mul / torch.arange(1,T+1).float().view(-1,1)
mul
xbow[0]

tensor([[-2.1126, -1.2052],
        [-0.9612, -0.0153],
        [ 0.2226,  0.6182],
        [ 0.0877,  0.6737],
        [-0.1936,  0.5601],
        [-0.0589,  0.3390],
        [-0.1765,  0.2706],
        [ 0.0742,  0.2018]], dtype=torch.float64)

tensor([[-2.1126, -1.2052],
        [-0.9612, -0.0153],
        [ 0.2226,  0.6182],
        [ 0.0877,  0.6737],
        [-0.1936,  0.5601],
        [-0.0589,  0.3390],
        [-0.1765,  0.2706],
        [ 0.0742,  0.2018]])

In [29]:
multiplier = torch.tril(torch.ones(T,T,dtype=torch.float64))
multiplier

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]], dtype=torch.float64)

putting all together

In [38]:
xbow2 = (torch.tril(torch.ones(T,T,dtype=torch.float64)) @ x) / torch.arange(1,T+1).view(-1,1).view(1,-1,1)
xbow[2], xbow2[2]
torch.all(xbow[2] == xbow2[2])
torch.all(xbow[2] - xbow2[2] < 1e-7)
torch.allclose(xbow, xbow2.float()) 

(tensor([[-0.5468,  0.7539],
         [-0.2377, -1.0792],
         [-0.0696, -0.7041],
         [-0.2162, -0.4762],
         [-0.2657, -0.4687],
         [-0.3456, -0.3903],
         [-0.3658, -0.4455],
         [-0.3455, -0.2466]]),
 tensor([[-0.5468,  0.7539],
         [-0.2377, -1.0792],
         [-0.0696, -0.7041],
         [-0.2162, -0.4762],
         [-0.2657, -0.4687],
         [-0.3456, -0.3903],
         [-0.3658, -0.4455],
         [-0.3455, -0.2466]], dtype=torch.float64))

tensor(False)

tensor(True)

True

Doing same thing using `softmax`

In [45]:
trill = torch.tril(torch.ones(T,T))
w = torch.zeros(T,T, dtype=torch.float64)
w2 = w.masked_fill(trill == 0, float('-inf'))
w3 = F.softmax(w2, dim=1) 
xbow3 = w3 @ x 
torch.allclose(xbow.float(), xbow3.float())

True

In [46]:
xbow[0] , xbow2[0] , xbow3[0]

(tensor([[-2.1126, -1.2052],
         [-0.9612, -0.0153],
         [ 0.2226,  0.6182],
         [ 0.0877,  0.6737],
         [-0.1936,  0.5601],
         [-0.0589,  0.3390],
         [-0.1765,  0.2706],
         [ 0.0742,  0.2018]]),
 tensor([[-2.1126, -1.2052],
         [-0.9612, -0.0153],
         [ 0.2226,  0.6182],
         [ 0.0877,  0.6737],
         [-0.1936,  0.5601],
         [-0.0589,  0.3390],
         [-0.1765,  0.2706],
         [ 0.0742,  0.2018]], dtype=torch.float64),
 tensor([[-2.1126, -1.2052],
         [-0.9612, -0.0153],
         [ 0.2226,  0.6182],
         [ 0.0877,  0.6737],
         [-0.1936,  0.5601],
         [-0.0589,  0.3390],
         [-0.1765,  0.2706],
         [ 0.0742,  0.2018]], dtype=torch.float64))

implies : 
* first row can't interact with future (2,8) 
* second row can interact with first col, but not with rest on right... (and so on)

In [48]:
w2

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float64)

> here init used is all zeroes, but in practice it will be real data !

# Debug

In [11]:
model_name = '04_02_better_usage_of_context'
'''
In this script, I used a better context for the model to generate text.
'''

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
from datetime import datetime

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'[>] Device : {device}')

device = 'cpu'


'\nIn this script, I used a better context for the model to generate text.\n'

[>] Device : cuda


In [12]:

train_model = False
train_from_scratch = False

alpha = 1e-4

batch_size = 64
block_size = 16
max_iters = 100000
pred_char_len = 1000
eval_iters = max_iters // 100

n_embeddings = 72 

models_folder = './saved_models'
os.makedirs(models_folder, exist_ok=True)

model_file = f'{models_folder}/{model_name}_{device}.pth'
encoding_file = f'{models_folder}/04_01_bi_gram_Generator_{device}_encoding.pth'
data_file = './data/wikitext/processed.txt'
chars_file = './data/wikitext/chars.txt'

model_exists = os.path.exists(model_file)
encoding_exists = os.path.exists(encoding_file)
data_exists = os.path.exists(data_file)
chars_exists = os.path.exists(chars_file)


if not data_exists:
    print(f'[!] Data not found !')
    exit(0)

data = None
if not chars_exists:
    print(f'[!] chars not found !')
    print(f'[>] Loading Data & Getting chars...')
    data = open(data_file, 'r').read()
    chars = ''.join(sorted(list(set(data))))
    open(chars_file, 'w').write(chars)
    chars_exists = True
    print(f'[+] Data Loaded and chars saved !')
else:
    chars = open(chars_file, 'r').read()


def enc(x, chars=chars):
    idxs = []
    for c in x:
        idxs.append(chars.index(c))
    return idxs


def dec(x, chars=chars):
    txt = ''
    for i in x:
        txt += chars[i]
    return txt


if encoding_exists:
    encoded_data = torch.load(encoding_file).clone().detach()
    encoded_data = encoded_data.to(device)
    print(f'[+] Encoding Loaded !')
else:
    print(f'[>] Loading Data ...')
    if data is None:
        data = open(data_file, 'r').read()
    print(f'[+] Data Loaded !')
    print(f'[>] Encoding Data ...')
    st = datetime.now()
    encoded_data = torch.tensor(enc(data), dtype=torch.long).to(device)
    et = datetime.now()
    print(f'[+] Data Encoded in {et - st} !')
    torch.save(encoded_data, encoding_file)
    print(f'[+] Encoding saved !')

vocab_size = len(chars)
print(f'[>] Number of Unique Characters : {vocab_size}')

n1 = int(0.8 * len(encoded_data))
n2 = int(0.9 * len(encoded_data))
Xtr = encoded_data[:n1]
Xdev = encoded_data[n1:n2]
Xte = encoded_data[n2:]

Xtr.to(device)
Xdev.to(device)
Xte.to(device)

[>] Loading Data ...
[+] Data Loaded !
[>] Encoding Data ...


KeyboardInterrupt: 

In [4]:
class BiGramLanguageModel(nn.Module):

    def __init__(self, vocab_size , block_size):
        super(BiGramLanguageModel, self).__init__()
        self.vocab_size = vocab_size
        self.block_size = block_size
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embeddings)
        self.position_embeddings = nn.Embedding(block_size, n_embeddings)
        self.lang_modelling_head = nn.Linear(n_embeddings, vocab_size)

    def forward(self, x, y=None):
        # (batch_size, block_size, vocab_size)
        print(f'x : {x.shape}')
        token_embeddings = self.token_embedding_table(x)
        print(f'token_embeddings : {token_embeddings.shape}')

        pos_embeddings = self.position_embeddings(torch.arange(x.shape[1], device=device))
        print(f'pos_embeddings : {pos_embeddings.shape}')
        embeds = token_embeddings + pos_embeddings
        print(f'embeds : {embeds.shape}')

        logits = self.lang_modelling_head(embeds)
        print(f'logits : {logits.shape}')
        if y is None:
            return logits

        # entropy expects : (N, C) input : (batch_size * block_size, vocab_size)
        loss = F.cross_entropy(logits.view(-1, self.vocab_size), y.view(-1))
        return logits, loss

    def generate(self, x, n_pred):
        for _ in range(n_pred):
            logits = self(x[:, -self.block_size])[:, -1, :]
            prob_dist = F.softmax(logits, -1)
            x = torch.cat([x, torch.multinomial(prob_dist, 1)], -1).to(device)
        return x


In [5]:

def get_batch(split):
    if split == 'train':
        X = Xtr
    elif split == 'dev':
        X = Xdev
    else:
        X = Xte
    start = np.random.randint(0, len(X) - block_size - 1, (batch_size,))
    X_batch = torch.stack([X[s:s + block_size] for s in start]).to(device)
    y_batch = torch.stack([X[s+1:s + block_size+1] for s in start]).to(device)
    return X_batch.to(device), y_batch.to(device)


@torch.no_grad()
def estimate_batch_loss(model):
    X_batch, y_batch = get_batch('dev')
    logits, loss = model(X_batch, y_batch)
    return loss.item()


@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'dev', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [ ]:

model = BiGramLanguageModel(vocab_size)
model = model.to(device)

if train_model:
    print(f'\n[>] Training ...')
    if not model_exists or train_from_scratch:
        print(f'[!] Training from scratch ...')
    else:
        model.load_state_dict(torch.load(model_file))
        print('[+] Model loaded!')

    st = datetime.now()
    optimiser = optim.AdamW(model.parameters(), lr=alpha)
    for iter in range(max_iters):
        x, y = get_batch('train')
        logits, loss = model(x, y)
        loss.backward()
        optimiser.step()
        optimiser.zero_grad(set_to_none=True)
        if iter % (max_iters // 10) == 0:
            train_loss = estimate_loss(model)['train']
            dev_loss = estimate_loss(model)['dev']
            print(f'Iter : {iter:7d}, Train Loss : {train_loss:.4f}, Valid Loss : {dev_loss:.4f}')
    et = datetime.now()
    print(f'[+] Training Done in {et - st} !')

    # Save the model
    torch.save(model.state_dict(), model_file)
    print('[+] Model saved!')

else:
    # Load the model
    model.load_state_dict(torch.load(model_file))
    print('[+] Model loaded!')

# model.to(device)


<All keys matched successfully>

[+] Model loaded!


In [ ]:

print(f'\n[>] Testing ...')
out = estimate_loss(model)
for ty, loss in out.items():
    print(f'\t{ty:5} Loss : {loss:.4f}')
print(f'\n')



[>] Testing ...
x : torch.Size([64, 16])
token_embeddings : torch.Size([64, 16, 72])
pos_embeddings : torch.Size([16, 72])
embeds : torch.Size([64, 16, 72])
x : torch.Size([64, 16])
token_embeddings : torch.Size([64, 16, 72])
pos_embeddings : torch.Size([16, 72])
embeds : torch.Size([64, 16, 72])
x : torch.Size([64, 16])
token_embeddings : torch.Size([64, 16, 72])
pos_embeddings : torch.Size([16, 72])
embeds : torch.Size([64, 16, 72])
x : torch.Size([64, 16])
token_embeddings : torch.Size([64, 16, 72])
pos_embeddings : torch.Size([16, 72])
embeds : torch.Size([64, 16, 72])
x : torch.Size([64, 16])
token_embeddings : torch.Size([64, 16, 72])
pos_embeddings : torch.Size([16, 72])
embeds : torch.Size([64, 16, 72])
x : torch.Size([64, 16])
token_embeddings : torch.Size([64, 16, 72])
pos_embeddings : torch.Size([16, 72])
embeds : torch.Size([64, 16, 72])
x : torch.Size([64, 16])
token_embeddings : torch.Size([64, 16, 72])
pos_embeddings : torch.Size([16, 72])
embeds : torch.Size([64, 16, 7

In [ ]:

print(f'[>] Generating ...')

print('-'*100 + '\n\tStarting with no context\n' + '-'*100)
print(dec(model.generate(torch.zeros((1, 1), dtype=torch.long,device=device), n_pred=pred_char_len)[0].tolist()))
print()


[>] Generating ...
----------------------------------------------------------------------------------------------------
	Starting with no context
----------------------------------------------------------------------------------------------------
x : torch.Size([1, 1])
token_embeddings : torch.Size([1, 1, 72])
pos_embeddings : torch.Size([1, 72])
embeds : torch.Size([1, 1, 72])
x : torch.Size([1, 2])
token_embeddings : torch.Size([1, 2, 72])
pos_embeddings : torch.Size([2, 72])
embeds : torch.Size([1, 2, 72])
x : torch.Size([1, 3])
token_embeddings : torch.Size([1, 3, 72])
pos_embeddings : torch.Size([3, 72])
embeds : torch.Size([1, 3, 72])
x : torch.Size([1, 4])
token_embeddings : torch.Size([1, 4, 72])
pos_embeddings : torch.Size([4, 72])
embeds : torch.Size([1, 4, 72])
x : torch.Size([1, 5])
token_embeddings : torch.Size([1, 5, 72])
pos_embeddings : torch.Size([5, 72])
embeds : torch.Size([1, 5, 72])
x : torch.Size([1, 6])
token_embeddings : torch.Size([1, 6, 72])
pos_embeddings : t

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling cublasLtMatmul with transpose_mat1 1 transpose_mat2 0 m 96 n 17 k 72 mat1_ld 72 mat2_ld 72 result_ld 96 abcType 0 computeType 68 scaleType 0

In [9]:
starts_with = " = Marvel = \n "
print('-'*100 + f'\n\tStarting with {starts_with}\n' + '-'*100)

contxt = torch.tensor(enc(starts_with), dtype=torch.long).unsqueeze(0).to(device)
print(dec(model.generate(contxt, n_pred=pred_char_len)[0].tolist()))
print()

----------------------------------------------------------------------------------------------------
	Starting with  = Marvel = 
 
----------------------------------------------------------------------------------------------------


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:

starts_with = " = Computer = \n "
print('-'*100, f'\n\tStarting with {starts_with}\n')
print('-'*100)

contxt = torch.tensor(enc(starts_with), dtype=torch.long).unsqueeze(0).to(device)
print(dec(model.generate(contxt, n_pred=pred_char_len)[0].tolist()))
print()

print('[+] hehe boi !')

""" 

Sample Output :



"""